# Pose Graph Optimization

In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import collections as mc
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import matplotlib.animation as animation
import copy
%matplotlib notebook
import sys, os
import shutil
import math
import time
from scipy.io import savemat,loadmat,whosmat
from scipy.special import psi
from scipy.stats.distributions import chi2

Import the SE-Sync Python library built using pybind

In [2]:
import PySESync
start=time.time()

In [3]:
name="CSAIL"
# name="input_g2o"
filename = "../../data/%s.g2o"%name

measurements, num_poses = PySESync.read_g2o_file(filename)
loop_clo=len(measurements)-num_poses
d = measurements[0].R.shape[0]

print("Loaded %d measurements between %d %d-dimensional poses from file %s" % (len(measurements), num_poses, d, filename))

Loaded 1172 measurements between 1045 2-dimensional poses from file ../../data/CSAIL.g2o


### Run Basic Solver: SE-Sync!

Set SE-Sync options

In [4]:
opts = PySESync.SESyncOpts()
opts.num_threads = 4
opts.verbose=False

opts.r0 = d
opts.formulation = PySESync.Formulation.Explicit # Options are Simplified or Explicit
opts.initialization = PySESync.Initialization.Random   # Options are Chordal or Random

# Termination criteria
opts.rel_func_decrease_tol = 1e-6
opts.min_eig_num_tol = 1e-3
opts.max_time = 900

opts.log_iterates = False


save_animation = False

In [5]:
result = PySESync.SESync(measurements, opts)
xhat = result.xhat

In [6]:
xhat = result.xhat
R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
tgt = np.matmul(R0inv, xhat[:, 0:num_poses])

In [7]:
measurements, num_poses = PySESync.read_g2o_file(filename)
shutil.copy("../../data/%s.g2o"%name, "../../data/%s2.g2o"%name)
shutil.copy("../../data/%s.g2o"%name, "../../data/%s3.g2o"%name)

'../../data/CSAIL3.g2o'

Load in some data from a .g2o file

In [8]:
##Function for data corruption
def data_corrupt(name,percent):
    file1 = open('../../data/%s.g2o'%name, 'r')

    lines = file1.readlines()
    file1.close()

    loop_clo=len(measurements)-num_poses
    c=0

    file2 = open('../../data/%s2.g2o'%name, 'w')

    for line in reversed(lines):
        if c<math.floor(loop_clo*percent):

            a=line.split()

            rander0=np.random.uniform(low=-5,high=5)
            rander1=np.random.uniform(low=-5,high=5)

            a[3]='%f'%rander0
            a[4]='%f'%rander1
            angle=np.pi    
            dthv=np.random.uniform(low=-angle, high=angle)
            a[5]='%f'%dthv    

            file2.write(" ".join(a))    
            file2.write("\n")

        else:
            file2.write(line)

        c+=1

    file2.close()    
    file3 = open('../../data/%s2.g2o'%name, 'r')
    lines2 = file3.readlines()
    file3.close()
    file4 = open('../../data/%s3.g2o'%name, 'w')

    for line in reversed(lines2):    
        file4.write(line)
        
    file4.close()    

In [9]:
percent=0.7
data_corrupt(name,percent)

In [10]:
filename = "../../data/%s3.g2o"%name

measurements, num_poses = PySESync.read_g2o_file(filename)
loop_clo=len(measurements)-num_poses
d = measurements[0].R.shape[0]

print("Loaded %d measurements between %d %d-dimensional poses from file %s" % (len(measurements), num_poses, d, filename))

Loaded 1172 measurements between 1045 2-dimensional poses from file ../../data/CSAIL3.g2o


Run SE-Sync!

In [11]:
tau=np.zeros(len(measurements))
kappa=np.zeros(len(measurements))

for j in range(len(measurements)):
    tau[j]=measurements[j].tau
    kappa[j]=measurements[j].kappa

In [12]:
## residual function
def r(xhat_l,measurements_l,d_l,tau_l,kappa_l,num_poses_l):
    residues=np.zeros(len(measurements))    
    
    xhat_lR=xhat_l[:,num_poses_l:]
    
    for ij in range(len(measurements_l)):
        Ri=xhat_lR[:,d*(measurements_l[ij].i):d*(measurements_l[ij].i)+1+1]
        Rj=xhat_lR[:,d*(measurements_l[ij].j):d*(measurements_l[ij].j)+1+1]
        Rij=measurements_l[ij].R
        tij=measurements_l[ij].t
        ti=xhat_l[:,measurements_l[ij].i]
        tj=xhat_l[:,measurements_l[ij].j]
        
        ti=np.reshape(ti,(2,1))
        tj=np.reshape(tj,(2,1))
        tij=np.reshape(tij,(2,1))
    
        norm1=np.linalg.norm(tj-ti-np.matmul(Ri,tij))        
        norm2=np.linalg.norm(Rj-np.matmul(Ri,Rij))            
    
        residues[ij]=tau[ij]*norm1*norm1+kappa[ij]*norm2*norm2
    
    return residues                  

In [13]:
# def areBinaryWeights(weights_l):
#     flag=True
#     th=10**(-12)
    
#     for i in range(len(weights_l)):
#         if weights_l[i]>th and weights_l[i]<1-th:
#             flag=False
#             break
#     return flag        

In [14]:
#GNC-TLS

def gncWeightsUpdate(mu_l,resi_l,eps_l,weights_l):
    th1=(mu_l+1)/mu_l*eps_l
    th2=mu_l/(mu_l+1)*eps_l
    poses=len(resi_l)-len(weights_l)
    
    for j in range(len(weights_l)):
        if resi_l[poses+j]-th1 >= 0:
            weights_l[j]=0
        elif resi_l[poses+j]-th2 <= 0:
            weights_l[j]=1
        else: 
            weights_l[j]=np.sqrt(eps_l*mu_l*(mu_l+1)/resi_l[poses+j])-mu_l
    
    return weights_l

def GNC_TLS(xhat,measurements,d,tau,kappa,num_poses,loop_clo,opts,tgt):
    start=time.time()
    dth=1
    eps=chi2.ppf(0.99, df=2)
    r_empty=r(xhat,measurements,d,tau,kappa,num_poses)
    r_m=sum(r_empty)
    tau_gns=max(r_empty)
    mu_GNS=eps
    mu_GNS=mu_GNS/(2*tau_gns-eps)
    t=0
    residues=r_empty
    w=np.ones(loop_clo)

    while dth>10**-5:
        t=t+1
        w=gncWeightsUpdate(mu_GNS,residues,eps,w)
    #     print(sum(w))
        wi=np.append(np.ones(num_poses),w)
        r_p=sum(np.multiply(wi,residues))
    #     print(r_p)

        for k in range(len(measurements)): 
            measurements[k].tau=tau[k]*wi[k]
            measurements[k].kappa=kappa[k]*wi[k]

        result = PySESync.SESync(measurements, opts)

        xhat = result.xhat
    #     print(xhat)

        mu_GNS=mu_GNS*1.4

        residues=r(xhat,measurements,d,tau,kappa,num_poses)

        dth=abs((r_p-r_m)/r_m)
        r_m=r_p
        
    R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
    t = np.matmul(R0inv, xhat[:, 0:num_poses])
    
    rmse=(np.sqrt(np.mean((tgt-t)**2)))
    
    end=time.time()
    time_elapsed=(end-start)
    
    return rmse,time_elapsed        

In [15]:
#GNC-GM

def gncWeightsUpdategm(mu_l,resi_l,eps_l,weights_l):
    poses=len(resi_l)-len(weights_l)
    
    for j in range(len(weights_l)):
        weights_l[j]=(1/(resi_l[poses+j]/(eps_l*mu_l)+1))**2
        
    return weights_l

def GNC_GM(xhat,measurements,d,tau,kappa,num_poses,loop_clo,opts,tgt):
    start=time.time()   
    dth=1
    eps=chi2.ppf(0.99, df=2)
    r_empty=r(xhat,measurements,d,tau,kappa,num_poses)
    r_m=sum(r_empty)
    tau_gm=max(r_empty)
    mu_GM=2*tau_gm/eps
    t=0
    residues=r_empty
    w=np.ones(loop_clo)
    
    while mu_GM>1:
        t=t+1
#         print(t)
#         print(mu_GM)
        w=gncWeightsUpdategm(mu_GM,residues,eps,w)
#         print(sum(w))
        wi=np.append(np.ones(num_poses),w)
        r_p=sum(np.multiply(wi,residues))

        for k in range(len(measurements)): 
            measurements[k].tau=tau[k]*wi[k]
            measurements[k].kappa=kappa[k]*wi[k]

    #     blockPrint()
        result = PySESync.SESync(measurements, opts)
    #     enablePrint()

        xhat = result.xhat
    #     print(xhat)

        mu_GM=mu_GM/1.4

        residues=r(xhat,measurements,d,tau,kappa,num_poses)

        dth=abs((r_p-r_m)/r_m)
        r_m=r_p
        
    R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
    t = np.matmul(R0inv, xhat[:, 0:num_poses])
    
    rmse=(np.sqrt(np.mean((tgt-t)**2)))
    
    end=time.time()
    time_elapsed=(end-start)
    
    return rmse,time_elapsed
        

In [16]:
##EROR
def rorWeightsUpdate(resi_l,weights_l):
    poses=len(resi_l)-len(weights_l)
    rmax=max(np.multiply(resi_l[poses:],weights_l))
    rmin=min(np.multiply(resi_l[poses:],weights_l))
#     rmax=max(np.multiply(resi_l,np.append(np.ones(poses),weights_l)))
#     rmin=min(np.multiply(resi_l,np.append(np.ones(poses),weights_l)))   
    
    v=(rmax+rmin)/2
    eps=chi2.ppf(0.99, df=2)
    v=max(v,eps)
    for j in range(len(weights_l)):
        weights_l[j]=1/((resi_l[poses+j])/v+1)

    return weights_l

def EROR(xhat,measurements,d,tau,kappa,num_poses,loop_clo,opts,tgt):
    start=time.time()   
    dth=1
    t=0
    residues=r(xhat,measurements,d,tau,kappa,num_poses)
    r_m=sum(residues)
    w=np.ones(loop_clo)
#     w=rorWeightsUpdate(residues,w)
    wi=np.append(np.ones(num_poses),w)
    eps=chi2.ppf(0.99, df=2)
    while dth>10**-5:

#     while max(np.multiply(residues[num_poses:],w))>10:
        t=t+1

        w=rorWeightsUpdate(residues,w)
        wi=np.append(np.ones(num_poses),w)
        r_p=sum(np.multiply(wi,residues))

        for k in range(len(measurements)): 
            measurements[k].tau=tau[k]*wi[k]
            measurements[k].kappa=kappa[k]*wi[k]

        result = PySESync.SESync(measurements, opts)
        xhat = result.xhat
        residues=r(xhat,measurements,d,tau,kappa,num_poses)


        dth=abs((r_p-r_m)/r_m)
        r_m=r_p

    R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
    t = np.matmul(R0inv, xhat[:, 0:num_poses])

    rmse=(np.sqrt(np.mean((tgt-t)**2)))

    end=time.time()
    time_elapsed=(end-start)
    
    return rmse,time_elapsed

In [17]:
##ESOR
def sorWeightsUpdate2(resi_l,weights_l):
    poses=len(resi_l)-len(weights_l)
    ravg=sum(np.multiply(resi_l[poses:],weights_l))+sum(resi_l[:poses-1])
    ravg=ravg/sum(np.append(np.ones(num_poses),weights_l))
    eps=chi2.ppf(0.99, df=2)
    ravg=max(ravg,eps)
    for j in range(len(weights_l)):
#         weights_l[j]=1/(1+np.exp(.5*(resi_l[poses+j]-ravg)))
        weights_l[j]=1/(1+np.exp(5*(resi_l[poses+j]-ravg)))
        
    return weights_l

def ESOR(xhat,measurements,d,tau,kappa,num_poses,loop_clo,opts,tgt):
    start=time.time()
    dth=1
    t=0
    residues=r(xhat,measurements,d,tau,kappa,num_poses)
    r_m=sum(residues)
    w=np.ones(loop_clo)
#     w=sorWeightsUpdate(residues,w)
    wi=np.append(np.ones(num_poses),w)
    eps=chi2.ppf(0.99, df=2)
    
    while dth>10**-5:    
    
        t=t+1
#         print(sum(w))

        w=sorWeightsUpdate2(residues,w)
        wi=np.append(np.ones(num_poses),w)
        r_p=sum(np.multiply(wi,residues))
        
        for k in range(len(measurements)): 
            measurements[k].tau=tau[k]*wi[k]
            measurements[k].kappa=kappa[k]*wi[k]

        result = PySESync.SESync(measurements, opts)
        xhat = result.xhat
        residues=r(xhat,measurements,d,tau,kappa,num_poses)

        dth=abs((r_p-r_m)/r_m)
        r_m=r_p
    

        
    R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
    t = np.matmul(R0inv, xhat[:, 0:num_poses])

    rmse=(np.sqrt(np.mean((tgt-t)**2)))

    end=time.time()
    time_elapsed=(end-start)
    
    return rmse,time_elapsed

In [18]:
##ASOR
def gammaOmegasUpdate(resi_l,weights_l,bin,a0):
    poses=len(resi_l)-len(weights_l)
    
    
    for j in range(len(weights_l)):
        weights_l[j]=1/(1+np.exp(.5*(resi_l[poses+j]))*(math.pi**-.5)*(bin**a0)/(.5*resi_l[poses+j]+bin)**(a0+.5)  )

    return weights_l

def fun_b(resi_l,weights_l,omegas_l,bin,a0):
    poses=len(resi_l)-len(weights_l)
    A=10000;
    B=1000;
    num=0;den=0;
    
    for j in range(len(weights_l)):
        
        num=num+(1-omegas_l[j])
        den=den+(1-omegas_l[j])/(.5*resi_l[poses+j]+bin)

    num=A+a0*num
    den=B+(a0+.5)
    
    bl=num/den
    return bl


def gammaWeightsUpdate(resi_l,weights_l,omegas_l,bin,a0):
    poses=len(resi_l)-len(weights_l)
    
    
    for j in range(len(weights_l)):
        weights_l[j]=omegas_l[j]+(1-omegas_l[j])*(a0+.5)/(.5*resi_l[poses+j]+bin)

    return weights_l

def ASOR(xhat,measurements,d,tau,kappa,num_poses,loop_clo,opts,tgt):
    start=time.time()
    b=10000
    a=.5
    dth=1
    t=0
    residues=r(xhat,measurements,d,tau,kappa,num_poses)
    r_m=sum(residues)
    w=np.ones(loop_clo)
#     w=sorWeightsUpdate(residues,w)
    wi=np.append(np.ones(num_poses),w)
    eps=chi2.ppf(0.99, df=2)
    
    while dth>10**-5:    
    

        t=t+1
#         print(sum(w))
#         w=sorWeightsUpdate(residues,w)

        omega=gammaOmegasUpdate(residues,w,b,a)
        b=fun_b(residues,w,omega,b,a)
        w=gammaWeightsUpdate(residues,w,omega,b,a)
        wi=np.append(np.ones(num_poses),w)
        r_p=sum(np.multiply(wi,residues))
        
        for k in range(len(measurements)): 
            measurements[k].tau=tau[k]*wi[k]
            measurements[k].kappa=kappa[k]*wi[k]

        result = PySESync.SESync(measurements, opts)
        xhat = result.xhat
        residues=r(xhat,measurements,d,tau,kappa,num_poses)

        dth=abs((r_p-r_m)/r_m)
        r_m=r_p
    

        
    R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + d])
    t = np.matmul(R0inv, xhat[:, 0:num_poses])

    rmse=(np.sqrt(np.mean((tgt-t)**2)))

    end=time.time()
    time_elapsed=(end-start)
    
    return rmse,time_elapsed

In [19]:
#Define MC runs and max outlier percentage
MC=1
outlier_max=9
T=[]

In [20]:
for ii in range(outlier_max):
# ii=8
# if ii==8:
    
    percent=(ii+1)*.1
    
    xhat_array=[]
    mea_array=[]
    
    for j in range(MC):
        
        result = PySESync.SESync(measurements, opts)
        xhat = result.xhat

        xhat_array.append(result.xhat)
        

        data_corrupt(name,percent)
        filename = "../../data/%s3.g2o"%name
        measurements, num_poses = PySESync.read_g2o_file(filename)
        
        mea_array.append(measurements)
        
        loop_clo=len(measurements)-num_poses
        d = measurements[0].R.shape[0]
        print("Loaded %d measurements between %d %d-dimensional poses from file %s" % (len(measurements), num_poses, d, filename))

    print(ii)   

    print('GNC-GM')

    for j in range(MC):
        print(j)
        rmse_GNC_GM,time_GNC_GM=GNC_GM(xhat_array[j],mea_array[j],d,tau,kappa,num_poses,loop_clo,opts,tgt)
        T.append([rmse_GNC_GM,0,percent,time_GNC_GM])

    print('GNC-TLS')
        
    for j in range(MC):
        print(j)
        rmse_GNC_TLS,time_GNC_TLS=GNC_TLS(xhat_array[j],mea_array[j],d,tau,kappa,num_poses,loop_clo,opts,tgt)
        T.append([rmse_GNC_TLS,1,percent,time_GNC_TLS])


    print('EROR')

    for j in range(MC):
        print(j)
        rmse_ROR,time_ROR=EROR(xhat_array[j],mea_array[j],d,tau,kappa,num_poses,loop_clo,opts,tgt)
        T.append([rmse_ROR,2,percent,time_ROR])

    print('ESOR')        

    for j in range(MC):
        print(j)
        rmse_SOR,time_SOR=ESOR(xhat_array[j],mea_array[j],d,tau,kappa,num_poses,loop_clo,opts,tgt)
        T.append([rmse_SOR,3,percent,time_SOR])

    print('ASOR')        

    for j in range(MC):
        print(j)
        rmse_gamma,time_gamma=ASOR(xhat_array[j],mea_array[j],d,tau,kappa,num_poses,loop_clo,opts,tgt)
        T.append([rmse_gamma,4,percent,time_gamma])

Loaded 1172 measurements between 1045 2-dimensional poses from file ../../data/CSAIL3.g2o
0
GNC-GM
0
GNC-TLS
0
EROR
0
ESOR
0


/tmp/ipykernel_6035/1229503771.py:10: RuntimeWarning: overflow encountered in exp
  weights_l[j]=1/(1+np.exp(5*(resi_l[poses+j]-ravg)))


ASOR
0


/tmp/ipykernel_6035/3472373243.py:7: RuntimeWarning: overflow encountered in exp
  weights_l[j]=1/(1+np.exp(.5*(resi_l[poses+j]))*(math.pi**-.5)*(bin**a0)/(.5*resi_l[poses+j]+bin)**(a0+.5)  )


KeyboardInterrupt: 

In [ ]:

# print(T)

In [ ]:
np.array(T)

In [ ]:
with open('ResTable_CSAIL.txt', 'w') as fp:
# with open('ResTable_intel.txt', 'w') as fp:
    for item in T:
        fp.write("%f %s %f %f\n" %(item[0],item[1],item[2],item[3]))
    print('Done')